In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import dask.dataframe as dd

In [4]:
import gc

In [5]:
import pickle

##### Global Variables ####

In [6]:
gc.collect()

294

In [7]:
BASE_DIR = '/sharedHDD/rohit/timeseries_learning/ubiquant/'
DATA_DIR = BASE_DIR+'data/parquet/'

In [8]:
input_chunk_length = 30
output_chunk_length = 1

In [9]:
features = [f'f_{i}' for i in range(300)]

#### Read Data #####

In [10]:
%%time
train_df = pd.read_parquet(DATA_DIR+'train_low_mem.parquet')

CPU times: user 8.71 s, sys: 14.1 s, total: 22.8 s
Wall time: 4.32 s


In [11]:
train_df.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float32(301), object(1), uint16(2)
memory usage: 3.6+ GB


In [13]:
investment_ids = train_df.investment_id.unique().tolist()

In [14]:
####### Filter out only Fold 1 data ###########

In [15]:
with open(BASE_DIR+'input/folds.pickle', 'rb') as f:
    folds = pickle.load(f)

In [16]:
folds.keys()

dict_keys([0, 1, 2, 3, 4])

##### Prepare data ######

In [17]:
all_features_columns = ['investment_id', 'target'] + features

In [18]:
def get_feature_array_train(rows):
    target_val = rows.target.values
    
    inp_vec = rows[all_features_columns].values
    
    return inp_vec,target_val

In [19]:
def sliding_windows(X,Y,seq_length,prediction_length):
    x = []
    y = []
    for i in range(len(X)-(prediction_length+seq_length)):
        _x = X[i:(i+seq_length),:].reshape(seq_length,-1)
#         _y = Y[(i+seq_length):(i+prediction_length+seq_length),:]
        _y = Y[i+seq_length]
        x.append(_x)
        y.append(_y)

    return x,y

In [20]:
def make_sequences(df):
    x = []
    y = []
    investment_ids = df.investment_id.unique()
    for iid in investment_ids:
        if iid not in to_exclude_ids:
            df_tmp = df[df.investment_id == iid].sort_values('time_id')
            tmp_X,tmp_Y = get_feature_array_train(df_tmp)
            _x,_y = sliding_windows(tmp_X,tmp_Y,input_chunk_length,output_chunk_length)
            x.extend(_x)
            y.extend(_y)
            
#             gc.collect()
        # else:
        #     print('Not including iid:', iid)
    
    return np.float32(np.array(x)),np.float32(np.array(y))
            
            
        

In [21]:
#### For now doing only for 1 fold ######

In [ ]:
for fold in folds.keys():
#     train_data_fold = {}
    print('Preparing training data for fold:', fold)
    train_indxs = folds[fold]['train']
    test_indxs = folds[fold]['test']
#     test_f_df = train_df[train_df.index.isin(test_indxs)].reset_index(drop=True)
    train_f_df = train_df[train_df.index.isin(train_indxs)].reset_index(drop=True)
    print(train_f_df.shape)
    count_df = train_f_df.groupby('investment_id',as_index=False).agg(c = ('time_id', len))
    to_exclude_ids = count_df[count_df.c<input_chunk_length+1].investment_id.unique().tolist()
    
    X_train, Y_train = make_sequences(train_f_df)
#     Y_train = Y_train.reshape(-1,1)
    print(X_train.shape,Y_train.shape)
    np.savez_compressed(BASE_DIR+'input/fold_train_'+str(fold),X_train=X_train,Y_train=Y_train )
#     train_data_fold['X_train'] = X_train
#     train_data_fold['Y_train'] = Y_train
#     with open(BASE_DIR+'input/fold_train_X_'+str(fold)+'.pickle', 'wb') as handle:
#         pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(BASE_DIR+'input/fold_train_Y_'+str(fold)+'.pickle', 'wb') as handle:
#         pickle.dump(Y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

    

Preparing training data for fold: 0
(531075, 304)
(421258, 30, 302) (421258,)
Preparing training data for fold: 1
(1053141, 304)
(942773, 30, 302) (942773,)
Preparing training data for fold: 2
(1575207, 304)
(1464649, 30, 302) (1464649,)
Preparing training data for fold: 3
(2097273, 304)
